In [1]:
%matplotlib ipympl
%load_ext line_profiler
import gym
import gym_watten
from gym_watten.envs.watten_env import Color, Value
import numpy as np
from time import sleep
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import pydot_ng as pydot
from io import BytesIO
import pickle

ModuleNotFoundError: No module named 'gym_watten.envs.watten_env'

In [2]:
env = gym.make("Watten-v0")

ImportError: module 'gym_watten.envs' has no attribute 'WattenEnv'

In [59]:
EPISODES = 75
MCTS_SIMS = 20
MEMORY_SIZE = 300000
SAMPLES_PER_STATE = 1

BATCH_SIZE = 256
TRAINING_LOOPS = 100
EXPLORATION = 1

EPSILON = 0.2
ALPHA = 0.8
OBJECTIVE_OPPONENT = False